In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("../data/ml-25m/ratings.csv")
movies = pd.read_csv("../data/ml-25m/movies.csv")

In [8]:

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [10]:
df.sort_values(by='timestamp',inplace=True)

In [11]:
grp_by_train = df.groupby('userId')

In [12]:
import random
random.sample(list(grp_by_train.groups),k=10)

[50012, 62876, 74145, 42828, 59121, 128494, 41924, 43979, 146559, 120644]

In [15]:

import sys
sys.path.insert(0, '../')
from src.preprocessing import *
from src.models import *

In [14]:
# groups return group name as the key and location indices as values.
data, mapping, inverse_mapping = map_column(df, col_name='movieId')

In [21]:

model = Recommender(
        vocab_size=len(mapping) + 2,
        lr=1e-4,
        dropout=0.3,
    )

In [ ]:
model_path = "/home/jenazzad/PycharmProjects/recommender_transformer/recommender_models/recommender.ckpt"

model = Recommender(
        vocab_size=len(mapping) + 2,
        lr=1e-4,
        dropout=0.3,
    )
model.eval()
model.load_state_dict(torch.load(model_path)["state_dict"])

In [19]:

movie_to_idx = {a:mapping[b] for a,  b in zip(movies.title.tolist(), movies.movieId.tolist()) if b in mapping}
idx_to_movie = {v:k for k, v in movie_to_idx.items()}

In [23]:
def predict(list_movies, model, movie_to_idx, idx_to_movie):
    ids = [PAD]*(120-len(list_movies)-1) + [movie_to_idx[a] for a in list_movies] + [MASK]
    src = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    
    with torch.no_grad():
        prediction = model(src)
        
    masked_pred = prediction[0, -1].numpy()
    
    sorted_predicted_ids = np.argsort(masked_pred).tolist()[::-1]
    sorted_predicted_ids = [a for a in sorted_predicted_ids if a not in ids]
    return [idx_to_movie[a] for a in sorted_predicted_ids[:10] if a in idx_to_movie]

# Adventure / Fantasy


In [25]:



list_movies = ["Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
               "Harry Potter and the Chamber of Secrets (2002)",
               "Harry Potter and the Prisoner of Azkaban (2004)",
               "Harry Potter and the Goblet of Fire (2005)"]
top_movie = predict(list_movies, model, movie_to_idx, idx_to_movie)
top_movie

['Drowning Pool, The (1975)',
 'Love on Sunday (2006)',
 'Fly By Night (2019)',
 'Hayride (2012)',
 'Skinned Deep (2004)',
 'World of Glory (1991)',
 'Ballerina (La mort du cygne) (1937)',
 'Looking for Grace (2015)',
 'Los Marziano (2011)',
 'Amusement (2008)']

# Comedy

In [26]:

list_movies = ["Zootopia (2016)",
               "Toy Story 3 (2010)",
               "Toy Story 4 (2019)",
               "Finding Nemo (2003)",
               "Ratatouille (2007)",
               "The Lego Movie (2014)",
               "Ghostbusters (a.k.a. Ghost Busters) (1984)",
               "Ace Ventura: When Nature Calls (1995)"]
top_movie = predict(list_movies, model, movie_to_idx, idx_to_movie)
top_movie

['The Long Recess (1972)',
 'Story of Floating Weeds, A (Ukikusa monogatari) (1934)',
 "Vampire's Kiss (1989)",
 'The Farewell (2019)',
 'This Is Where I Leave You (2014)',
 'Big Short, The (2015)',
 'Gospel, The (2005)',
 "Remi Nobody's Boy (2018)",
 'Amusement (2008)',
 'Love on Sunday (2006)']